In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/dialnd/imbalanced-algorithms.git

Cloning into 'imbalanced-algorithms'...
remote: Enumerating objects: 428, done.
remote: Total 428 (delta 0), reused 0 (delta 0), pack-reused 428
Receiving objects: 100% (428/428), 28.65 MiB | 27.70 MiB/s, done.
Resolving deltas: 100% (211/211), done.


# Preparation

In [ ]:
import pandas as pd, numpy as np
from plotly import express as px
#  pip install -U kaleido

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/CMG - Crystal Prediction Project/Ternary Materials Point Group Prediction/Data/NOMAD_2/Classification_Data_8.pkl").dropna()
print(df.shape)
df.head()

(1381099, 24)


,Atomic Number 1,Atomic Number 2,Atomic Number 3,Element_1,Element_2,Element_3,Coefficient 1,Coefficient 2,Coefficient 3,formula_reduced,bravais_lattice,crystal_system,space_group_number,point_group,lattice_parameters,Oxidation 1,Oxidation 2,Oxidation 3,IonicRadius_1,IonicRadius_2,IonicRadius_3,IonizationPot1st_1,IonizationPot1st_2,IonizationPot1st_3
0,3,5,41,Li,B,Nb,4,1,1,BLi4Nb,cF,cubic,216,-43m,"{'a': 6.71860936e-10, 'b': 6.71860936e-10, 'c'...",1,-1,-3,0.755440,1.048943,1.842691,5.3917,8.298,6.7589
1,4,5,41,Be,B,Nb,4,1,1,BBe4Nb,cF,cubic,216,-43m,"{'a': 6.119642980000002e-10, 'b': 6.1196429800...",1,-1,-3,0.643658,1.048943,1.842691,9.3226,8.298,6.7589
2,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,cF,cubic,216,-43m,"{'a': 5.458027e-10, 'b': 5.458027e-10, 'c': 5....",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589
3,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,cF,cubic,225,m-3m,"{'a': 5.420156379999999e-10, 'b': 5.4201563799...",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589
4,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,mS,monoclinic,12,2/m,"{'a': 4.660283620000524e-10, 'b': 8.41781306e-...",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589


Configure

In [ ]:
FEATURE_NAMES = [
#     "Atomic Number 1","Atomic Number 2","Atomic Number 3",
    "Coefficient 1", "Coefficient 2", "Coefficient 3", 
    "IonizationPot1st_1", "IonizationPot1st_2", "IonizationPot1st_3", 
    "Oxidation 1", "Oxidation 2", "Oxidation 3", # Used to be electronegativity 
    "IonicRadius_1", "IonicRadius_2", "IonicRadius_3"
]

GROUPERS = [
    "Atomic Number 1","Atomic Number 2","Atomic Number 3",
    "Coefficient 1", "Coefficient 2", "Coefficient 3"
]

# Y_NAME = 'crystal_system'
# Y_NAME = 'bravais_lattice'
# Y_NAME = 'crystal_system'
Y_NAME = 'space_group_number'

Filter

In [ ]:
Y_NAME = 'space_group_number'
df[Y_NAME] = df[Y_NAME].astype(int).astype(str)
accepted_classes = (df[Y_NAME].value_counts() > 300).replace(False, np.nan).dropna().index.tolist()
df_filtered = df[df[Y_NAME].isin(accepted_classes)].copy()

print("Previous Data Size:", df.shape[0])
print("New Data Size:", df_filtered.shape[0])
print("Number of Classes Left:", len(accepted_classes))

Previous Data Size: 1381099
New Data Size: 1372271
Number of Classes Left: 50


In [ ]:
df_main = df_filtered[FEATURE_NAMES+[Y_NAME]].drop_duplicates().copy()

In [ ]:
df_main.head()

,Coefficient 1,Coefficient 2,Coefficient 3,IonizationPot1st_1,IonizationPot1st_2,IonizationPot1st_3,Oxidation 1,Oxidation 2,Oxidation 3,IonicRadius_1,IonicRadius_2,IonicRadius_3,space_group_number
0,4,1,1,5.3917,8.298,6.7589,1,-1,-3,0.755440,1.048943,1.842691,216
1,4,1,1,9.3226,8.298,6.7589,1,-1,-3,0.643658,1.048943,1.842691,216
2,2,1,1,9.3226,8.298,6.7589,2,-1,-3,0.452080,1.048943,1.842691,216
3,2,1,1,9.3226,8.298,6.7589,2,-1,-3,0.452080,1.048943,1.842691,225
4,2,1,1,9.3226,8.298,6.7589,2,-1,-3,0.452080,1.048943,1.842691,12


In [ ]:
df_grouped = df_main.groupby(FEATURE_NAMES)[Y_NAME].apply(list).reset_index()

# ClusterCentroid+RFC (Base-Line)

In [ ]:
y = df_grouped[Y_NAME]
X = df_grouped[FEATURE_NAMES]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# scaler = MinMaxScaler()
# scaler.fit(X_train.values)
# X_train_sc = pd.DataFrame(scaler.transform(X_train.values), columns = X_train.columns, index=X_train.index)
# X_test_sc =  pd.DataFrame(scaler.transform(X_test.values), columns = X_test.columns, index=X_test.index)

df_train = X_train.merge(y_train, left_index=True, right_index=True).explode(Y_NAME).reset_index().copy()
df_test = X_test.merge(y_test, left_index=True, right_index=True).copy()

In [ ]:
y_counts = df_train[Y_NAME].value_counts()
us_strategy = dict([
                    (y, size) if size < 50000 else (y, 50000) 
                    for (y, size) in zip(y_counts.index, y_counts.values)
                    ])

os_strategy = dict([
                    (y, size*7) if size < 3000 else (y, size) 
                    for (y, size) in us_strategy.items()
                    ])

In [ ]:
print(os_strategy)

{'216': 50000, '225': 50000, '221': 50000, '166': 50000, '123': 50000, '65': 50000, '99': 50000, '12': 50000, '10': 50000, '119': 48932, '47': 48595, '8': 48490, '139': 46766, '44': 44281, '35': 41061, '6': 40041, '71': 35963, '25': 28749, '107': 18591, '156': 18014, '160': 15347, '42': 13926, '38': 12181, '187': 6848, '62': 19607, '14': 11970, '227': 11158, '63': 9296, '194': 9191, '148': 8561, '15': 8232, '2': 8057, '1': 6125, '69': 5964, '164': 5831, '55': 4711, '129': 4669, '189': 4368, '11': 3885, '141': 3150, '186': 3108, '191': 2842, '127': 2618, '59': 2576, '193': 2100, '36': 2100, '74': 1960, '140': 1946, '4': 1855, '33': 1771}


In [ ]:
y_train = df_train[Y_NAME]
X_train = df_train[FEATURE_NAMES]

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

rus = RandomUnderSampler(random_state=0, sampling_strategy=us_strategy, replacement=True)
ros = RandomOverSampler(random_state=0, sampling_strategy=os_strategy)

X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
X_resampled, y_resampled = ros.fit_resample(X_resampled, y_resampled)

In [ ]:
# from imblearn.under_sampling import ClusterCentroids
# cc = ClusterCentroids(random_state=0, sampling_strategy=os_strategy)
# X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

In [ ]:
# from imblearn.over_sampling import SMOTE


# # ml_binner = MultiLabelBinarizer()
# # ml_binner.fit(df_grouped[Y_NAME])
# # y = ml_binner.transform(df_grouped[Y_NAME])
# # X = df_grouped[FEATURE_NAMES].values


# smote = SMOTE(sampling_strategy=os_strategy)
# X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
df_train_os = X_resampled.merge(y_resampled, left_index=True, right_index=True)
df_grouped = df_train_os.groupby(FEATURE_NAMES)[Y_NAME].apply(list).reset_index()


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
ml_binner = MultiLabelBinarizer()
ml_binner.fit(df_grouped[Y_NAME])


y_train_os = ml_binner.transform(df_grouped[Y_NAME])
X_train_os = df_grouped[FEATURE_NAMES]

y_test_f = ml_binner.transform(df_test[Y_NAME])
X_test_f = df_test[FEATURE_NAMES]

In [ ]:
import pickle 

# pickle.dump(X_train_os, open("util/X_train_os.pkl", 'wb'))
# pickle.dump(y_train_os, open("util/y_train_os.pkl", 'wb'))
# pickle.dump(X_test_f, open("util/X_test_f.pkl", 'wb'))
# pickle.dump(y_test_f, open("util/y_test_f.pkl", 'wb'))
# pickle.dump(ml_binner, open("util/ml_binner.pkl", 'wb'))

In [ ]:
X_train_os = pickle.load(open("util/X_train_os.pkl", 'rb'))
y_train_os = pickle.load(open("util/y_train_os.pkl", 'rb'))
X_test_f = pickle.load(open("util/X_test_f.pkl", 'rb'))
y_test_f = pickle.load(open("util/y_test_f.pkl", 'rb'))
ml_binner = pickle.load(open("util/ml_binner.pkl", 'rb'))

In [ ]:
y_test_f.shape
y_train_os.shape

(155388, 50)

# Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50, random_state=0)
clf.fit(X_train_os, y_train_os)

RandomForestClassifier(n_estimators=50, random_state=0)

In [ ]:
# import pickle
# pickle.dump(clf, open("rfc.pkl", 'wb'))


In [ ]:
X_test_f
X_train_os

y_train_os.shape
y_test_f.shape

,Coefficient 1,Coefficient 2,Coefficient 3,IonizationPot1st_1,IonizationPot1st_2,IonizationPot1st_3,Oxidation 1,Oxidation 2,Oxidation 3,IonicRadius_1,IonicRadius_2,IonicRadius_3
0,0.000000,0.000000,0.000000,0.033026,0.097406,0.163679,0.333333,0.583333,0.333333,0.797935,0.574702,0.779060
1,0.000000,0.000000,0.000000,0.033026,0.097406,0.250952,0.333333,0.583333,0.333333,0.797935,0.574702,0.862370
2,0.000000,0.000000,0.000000,0.033026,0.097406,0.294636,0.333333,0.583333,0.333333,0.797935,0.574702,0.589727
3,0.000000,0.000000,0.000000,0.033026,0.097406,0.295375,0.333333,0.583333,0.333333,0.797935,0.574702,0.626416
4,0.000000,0.000000,0.000000,0.033026,0.097406,0.355247,0.333333,0.583333,0.333333,0.797935,0.574702,0.595960
...,...,...,...,...,...,...,...,...,...,...,...,...
376601,0.870968,0.333333,0.086957,0.154625,0.283282,0.116055,0.333333,0.583333,0.750000,0.616641,0.281082,0.296010
376602,0.903226,0.333333,0.260870,0.718772,0.302028,0.260391,0.333333,0.500000,0.666667,0.513121,0.405402,0.337987
376603,0.935484,0.285714,0.304348,0.487312,0.283282,0.376956,0.500000,0.250000,0.250000,0.393236,0.571484,0.691813
376604,0.967742,0.238095,0.260870,1.000000,0.202529,0.272180,0.333333,0.750000,0.500000,0.576754,0.278700,0.448224


In [ ]:
y_pred = clf.predict(X_test_f)


In [ ]:
X_test_f.shape

(64128, 12)

In [ ]:
y_train_os.shape
y_test_f.shape

(64128, 50)

In [ ]:
from sklearn.metrics import classification_report
target_names = ml_binner.classes_
print(classification_report(y_test_f, y_pred, target_names=[str(i) for i in target_names]))

              precision    recall  f1-score   support

           1       0.67      0.19      0.29       223
          10       0.95      0.67      0.78     12553
         107       0.99      0.76      0.86      4688
          11       0.81      0.19      0.31       114
         119       0.96      0.66      0.78     12325
          12       0.92      0.61      0.74     12619
         123       0.99      0.67      0.80     13785
         127       0.89      0.40      0.55       102
         129       0.70      0.19      0.30       162
         139       0.91      0.59      0.71     11762
          14       0.66      0.24      0.35       427
         140       0.88      0.37      0.52        79
         141       0.65      0.27      0.38       110
         148       0.73      0.49      0.59       274
          15       0.76      0.33      0.46       290
         156       0.98      0.73      0.83      4550
         160       0.39      0.04      0.07      3823
         164       0.70    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
target_names = ml_binner.classes_
print(classification_report(y_test_f, y_pred, target_names=[str(i) for i in target_names]))

              precision    recall  f1-score   support

           1       0.67      0.27      0.39       223
          10       0.93      0.09      0.17     12553
         107       0.99      0.95      0.97      4688
          11       0.65      0.23      0.34       114
         119       0.93      0.10      0.19     12325
          12       0.91      0.09      0.16     12619
         123       0.97      0.09      0.16     13785
         127       0.73      0.59      0.65       102
         129       0.69      0.30      0.42       162
         139       0.91      0.16      0.27     11762
          14       0.63      0.33      0.43       427
         140       0.84      0.41      0.55        79
         141       0.69      0.44      0.53       110
         148       0.61      0.84      0.70       274
          15       0.68      0.41      0.51       290
         156       0.97      0.93      0.95      4550
         160       0.37      0.13      0.19      3823
         164       0.72    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.

